<a href="https://colab.research.google.com/github/andrewshawl/Vuelos/blob/main/Vuelos_AS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
pip install requests


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
import pandas as pd
import requests 
import threading

In [57]:
# Load the data
tickets = pd.read_csv('/content/drive/MyDrive/challenge_dataset.csv',  parse_dates=True)

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
tickets_reduced= tickets[["origin","destination","flight_num"]]

In [60]:
tickets_reduced=tickets_reduced.drop_duplicates(subset="flight_num", keep= "last")

In [61]:
ciudades_dest = [x for x in tickets["destination"]]
ciudades_origen = [x for x in tickets["origin"]]
ciudades = ciudades_dest + ciudades_origen
ciudades = list( dict.fromkeys(ciudades) )




In [62]:
def temp_de_ciudades(ciudades_i, ciudades_latlon):
  for ciudad in ciudades_i:
    if ciudad in ciudades_origen:
      lat=tickets[tickets["origin"] == ciudad ]["origin_latitude"].iloc[0]
      lon= tickets[tickets["origin"] == ciudad ]["origin_longitude"].iloc[0] 
    else:
      lat = tickets[tickets["destination"] == ciudad ]["destination_latitude"].iloc[0]
      lon= tickets[tickets["destination"] == ciudad ]["destination_longitude"].iloc[0]
  
    
    url="http://api.openweathermap.org/data/2.5/forecast?" + \
      "lat=" + str(lat)+ "&lon=" + str(lon)+"&cnt=5" + \
      "&appid=" + "d1a4fab2d46987ad898458389baa88a2"

    try:
      response= requests.get(url)
    except Exception as e:
      print(f"Error getting temp for url:{url}.\n Skipping")


    weather_prev = response.json()

    # La API da la temperatura cada 3 horas, asi que el 5to elemento son 12 PM
    weather = weather_prev["list"][4]


    temp = weather["main"]["temp"]
    ciudades_latlon[ciudad] = (temp)








In [63]:
ciudades_latlon = {}
num_threads = 4
paso = len(ciudades) // num_threads
modulo = len(ciudades) % num_threads

# Si la division deja un residuo, agregaos un thread extra
if modulo > 0:
  num_threads += 1
threads = [None] * num_threads


print(f'Haciendo las llamadas al api de openwather en {num_threads} procesos, '
  f'con {paso} ciudades por proceso')



for i in range(num_threads):
    if i == num_threads - 1 and modulo > 0:
      ciudades_i = ciudades[i*paso:(i*paso) + modulo]
    else:
      ciudades_i = ciudades[i*paso:(i+1)*paso]

    
    # print(ciudades_temp)
    threads[i] = threading.Thread(target=temp_de_ciudades,
                                  args=(ciudades_i,
                                        ciudades_latlon))
    threads[i].start()

for i in range(num_threads):
    threads[i].join()


ciudades_latlon


Haciendo las llamadas al api de openwather en 4 procesos, con 13 ciudades por proceso


{'MTY': 307.25,
 'MZT': 303.85,
 'ZCL': 295.71,
 'TLC': 292.68,
 'AMS': 295.7,
 'GUA': 297.57,
 'OAX': 299.34,
 'MEX': 296.26,
 'ATL': 298.18,
 'VSA': 308.74,
 'HUX': 304.73,
 'TAM': 304.65,
 'BZE': 303.46,
 'CEN': 305.92,
 'ZIH': 303.8,
 'GDL': 299.28,
 'MAD': 307.61,
 'DFW': 297.55,
 'PVR': 305.32,
 'CJS': 300.9,
 'ORD': 299.41,
 'SCL': 292.76,
 'LIM': 291.08,
 'PHX': 306.27,
 'CUN': 304.88,
 'AGU': 298.67,
 'HAV': 306.49,
 'PHL': 303.52,
 'CZM': 303.19,
 'TIJ': 299.59,
 'BOG': 289.67,
 'CUU': 300.16,
 'HMO': 304.02,
 'CLT': 302.23,
 'MIA': 304.81,
 'QRO': 297.44,
 'YYZ': 297.67,
 'CME': 302.28,
 'LAX': 296.21,
 'JFK': 298.97,
 'MID': 309.47,
 'BJX': 296.9,
 'IAH': 304.35,
 'CTM': 303.39,
 'TRC': 303.91,
 'YVR': 292.75,
 'PBC': 295.34,
 'VER': 303.28,
 'SLP': 297.51,
 'PXM': 303.85,
 'CDG': 298.94,
 'ACA': 300.69}

In [64]:
temp_origin = [ciudades_latlon[x] for x in tickets_reduced["origin"] ]
temp_destiny=[ciudades_latlon[x] for x in tickets_reduced["destination"] ]

tickets_reduced["origin_weather"]=temp_origin
tickets_reduced['destination_weather']=temp_destiny 
tickets_reduced

,origin,destination,flight_num,origin_weather,destination_weather
1,MTY,TLC,119,307.25,292.68
2,MEX,MTY,2100,296.26,307.25
3,MTY,MEX,2101,307.25,296.26
9,MEX,MTY,2102,296.26,307.25
10,MTY,MEX,2103,307.25,296.26
...,...,...,...,...,...
2915,GDL,TIJ,110,299.28,299.59
2916,MEX,GDL,1104,296.26,299.28
2918,MEX,GDL,1120,296.26,299.28
2919,GDL,MEX,1127,299.28,296.26
